In [60]:
# A simple predict the next character by looking at last k characters

In [1]:
!wget https://objectstore.e2enetworks.net/ai4b-public-nlu-nlg/indic-corp-frozen-for-the-paper-oct-2022/mr.txt

--2024-07-08 16:08:50--  https://objectstore.e2enetworks.net/ai4b-public-nlu-nlg/indic-corp-frozen-for-the-paper-oct-2022/mr.txt
Resolving objectstore.e2enetworks.net (objectstore.e2enetworks.net)... 164.52.210.97, 101.53.152.33, 164.52.206.154, ...
Connecting to objectstore.e2enetworks.net (objectstore.e2enetworks.net)|164.52.210.97|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14553660884 (14G) [text/plain]
Saving to: ‘mr.txt’

mr.txt                0%[                    ]       0  --.-KB/s               ^C


In [2]:
# Load the dataset

k = 10000
input_file_path = './data/mr.txt'
output_file_path = f"./data/mr_{k}.txt"

# Function to read the first k lines from the input file and write them to the output file
def read_and_write_first_k_lines(input_file, output_file, num_lines=1000):
    try:
        with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
            for i in range(num_lines):
                line = infile.readline()
                if not line:  # End of file reached before 1000 lines
                    break
                outfile.write(line)
        print(f"Successfully wrote the first {num_lines} lines to {output_file}.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the function
read_and_write_first_k_lines(input_file_path, output_file_path, k)

data_file = output_file_path
with open(data_file, 'r') as file:
    lines = file.readlines()

Successfully wrote the first 10000 lines to ./data/mr_10000.txt.


In [3]:
# Let's build a vocabulary
context = 3
vocab = set()

for line in lines:
    if line.strip() != "":
        # words = line.split(" ")
        # Remove blank words
        # words = [word.strip() for word in words if word.strip() != "" ]
        line = '<sos>' + line.strip() + '<eos>'
        for ch in line:
            vocab.add(ch)
vocab = list(vocab)
vocab_size = len(vocab)
print(vocab_size)

245


In [4]:
word_to_i = {word: i for i, word in enumerate(vocab)}
i_to_word = {i: word for word, i in word_to_i.items()}
print(word_to_i['.'])

139


In [5]:
# Let's get the training dataset
from collections import deque

xs = []
ys = []

def get_queue():
    qu = deque(maxlen=context)
    for _ in range(context):
        qu.append('.')
    return qu

for line in lines:
    qu = get_queue()
    if line.strip() != "":
        line = line.strip()
        qu.append(line[0])
        for ch in line[1:]:
            xs.append(list(qu))
            ys.append(ch)
            qu.append(ch)
print(lines[0])
print(len(xs))
# for x, y in zip(xs, ys):
#     print(x, y)

ऊती संवर्धन तंत्राचे अनेक उपयोग आहेत. या तंत्राचा उपयोग विशेषकरून जीवशास्त्र व वैद्यकशास्त्रात होतो. वयोवृद्धी, पोषण, लसनिर्मिती, जन्मजात रोगांचे निदान, इंद्रियांचे रोपण, कर्करोग संशोधन व गर्भपोषण या क्षेत्रांत ऊती संवर्धन तंत्र प्रामुख्याने वापरले जाते. पेशींच्या चयापचयावर एखाद्या घटकाचा परिणाम पाहणे, सामान्य किंवा कर्करोगाच्या पेशींवर औषधांचा होणारा परिणाम पाहणे, प्रयोगशाळेत त्वचा तयार करणे इ. बाबी ऊती संवर्धनामुळे शक्य झाल्या आहेत. भाजलेल्या रुग्णाच्या त्वचारोपणासाठी ऊती संवर्धनाद्वारे निर्माण केलेली त्वचा वापरली जाते.

1418702


In [6]:
import torch

xs_enc = []
ys_enc = []
for inp, out in zip(xs, ys):
    temp = []
    for i in inp:
        temp.append(word_to_i[i])
    xs_enc.append(temp)
    ys_enc.append(word_to_i[out])
xs_enc = torch.tensor(xs_enc)
ys_enc = torch.tensor(ys_enc)

In [7]:
print(xs_enc.shape)
print(ys_enc.shape)
num_examples = len(xs)

torch.Size([1418702, 3])
torch.Size([1418702])


In [8]:
# Lookup table
lookup_table = torch.randn((vocab_size, 2), requires_grad=True)
lookup_table.shape

torch.Size([245, 2])

In [50]:
embeddings = lookup_table[xs_enc]
embeddings.shape

torch.Size([1418702, 3, 2])

In [51]:
# Define weights and biases
w1 = torch.randn((6, 100), requires_grad=True)
b1 = torch.randn(100, requires_grad=True)

w2 = torch.randn((100, vocab_size), requires_grad=True)
b2 = torch.randn(vocab_size, requires_grad=True)

In [52]:
# Forward pass
n1 = embeddings.view(-1, 6) @ w1 + b1
h1 = torch.tanh(n1)
logits = h1 @ w2 + b2
logits.shape

torch.Size([1418702, 245])

In [53]:
counts = logits.exp()
prob = counts / counts.sum(1, keepdims=True)
loss = -prob[torch.arange(num_examples), ys_enc].log().mean()
print(loss.item())

22.858861923217773


In [54]:
import torch.nn.functional as F

loss = F.cross_entropy(logits, ys_enc)
loss.item()

22.858861923217773

In [55]:
# Let's implement loop
epoch = 10

lookup_table = torch.randn((vocab_size, 2), requires_grad=True)
w1 = torch.randn((6, 100), requires_grad=True)
b1 = torch.randn(100, requires_grad=True)
w2 = torch.randn((100, vocab_size), requires_grad=True)
b2 = torch.randn(vocab_size, requires_grad=True)

parameters = [lookup_table, w1, b1, w2, b2]

for p in parameters:
    p.requires_grad = True
    
for _ in range(epoch):
    embeddings = lookup_table[xs_enc]
    n1 = embeddings.view(-1, 6) @ w1 + b1
    h1 = torch.tanh(n1)
    logits = h1 @ w2 + b2
    loss = F.cross_entropy(logits, ys_enc)
    loss.backward()
    for p in parameters:
        p.data += -0.1 * p.grad
        p.grad = None
    print(loss)

tensor(22.8625, grad_fn=<NllLossBackward0>)
tensor(22.3352, grad_fn=<NllLossBackward0>)
tensor(21.8611, grad_fn=<NllLossBackward0>)
tensor(21.4195, grad_fn=<NllLossBackward0>)
tensor(20.9986, grad_fn=<NllLossBackward0>)
tensor(20.5924, grad_fn=<NllLossBackward0>)
tensor(20.1982, grad_fn=<NllLossBackward0>)
tensor(19.8146, grad_fn=<NllLossBackward0>)
tensor(19.4405, grad_fn=<NllLossBackward0>)
tensor(19.0752, grad_fn=<NllLossBackward0>)


In [70]:
# Let's try to batchL on my own
import math

batch_size = 14187
num_batches = math.floor(len(ys_enc) / batch_size)
for _ in range(epoch):
    for i in range(num_batches-1):
        offset_start = i * 14187
        offset_end = (i+1) * 14187
        xs_batch = xs_enc[offset_start:offset_end]
        ys_batch = ys_enc[offset_start:offset_end]
        embeddings = lookup_table[xs_batch]
        n1 = embeddings.view(-1, 6) @ w1 + b1
        h1 = torch.tanh(n1)
        logits = h1 @ w2 + b2
        loss = F.cross_entropy(logits, ys_batch)
        loss.backward()
        for p in parameters:
            p.data += -0.1 * p.grad
            p.grad = None
        print(loss)

tensor(18.6823, grad_fn=<NllLossBackward0>)
tensor(18.3538, grad_fn=<NllLossBackward0>)
tensor(18.1245, grad_fn=<NllLossBackward0>)
tensor(17.7852, grad_fn=<NllLossBackward0>)
tensor(17.3629, grad_fn=<NllLossBackward0>)
tensor(17.2167, grad_fn=<NllLossBackward0>)
tensor(16.6701, grad_fn=<NllLossBackward0>)
tensor(16.4135, grad_fn=<NllLossBackward0>)
tensor(15.9931, grad_fn=<NllLossBackward0>)
tensor(15.6859, grad_fn=<NllLossBackward0>)
tensor(15.3667, grad_fn=<NllLossBackward0>)
tensor(15.2032, grad_fn=<NllLossBackward0>)
tensor(15.0301, grad_fn=<NllLossBackward0>)
tensor(14.8159, grad_fn=<NllLossBackward0>)
tensor(14.4873, grad_fn=<NllLossBackward0>)
tensor(14.2414, grad_fn=<NllLossBackward0>)
tensor(13.9645, grad_fn=<NllLossBackward0>)
tensor(13.7000, grad_fn=<NllLossBackward0>)
tensor(13.4302, grad_fn=<NllLossBackward0>)
tensor(13.2094, grad_fn=<NllLossBackward0>)
tensor(13.1689, grad_fn=<NllLossBackward0>)
tensor(12.8370, grad_fn=<NllLossBackward0>)
tensor(12.6738, grad_fn=<NllLoss

In [84]:
# Mini batching using andrej's approach

# Let's implement loop
epoch = 10000
batch_size = 1000
# lookup_table = torch.randn((vocab_size, 2), requires_grad=True)
# w1 = torch.randn((6, 100), requires_grad=True)
# b1 = torch.randn(100, requires_grad=True)
# w2 = torch.randn((100, vocab_size), requires_grad=True)
# b2 = torch.randn(vocab_size, requires_grad=True)

parameters = [lookup_table, w1, b1, w2, b2]

for p in parameters:
    p.requires_grad = True
    
for _ in range(epoch):
    sampling = torch.randint(0, xs_enc.shape[0], (batch_size,))
    xs_batch = xs_enc[sampling]
    ys_batch = ys_enc[sampling]

    embeddings = lookup_table[xs_batch]
    n1 = embeddings.view(-1, 6) @ w1 + b1
    h1 = torch.tanh(n1)
    logits = h1 @ w2 + b2
    loss = F.cross_entropy(logits, ys_batch)
    loss.backward()
    for p in parameters:
        p.data += -0.01 * p.grad
        p.grad = None
    print(loss)

tensor(2.8051, grad_fn=<NllLossBackward0>)
tensor(2.8099, grad_fn=<NllLossBackward0>)
tensor(2.7809, grad_fn=<NllLossBackward0>)
tensor(2.7912, grad_fn=<NllLossBackward0>)
tensor(2.7711, grad_fn=<NllLossBackward0>)
tensor(2.8321, grad_fn=<NllLossBackward0>)
tensor(2.8057, grad_fn=<NllLossBackward0>)
tensor(2.9216, grad_fn=<NllLossBackward0>)
tensor(2.7592, grad_fn=<NllLossBackward0>)
tensor(2.7895, grad_fn=<NllLossBackward0>)
tensor(2.8487, grad_fn=<NllLossBackward0>)
tensor(2.8161, grad_fn=<NllLossBackward0>)
tensor(2.9543, grad_fn=<NllLossBackward0>)
tensor(2.7717, grad_fn=<NllLossBackward0>)
tensor(2.8513, grad_fn=<NllLossBackward0>)
tensor(2.7864, grad_fn=<NllLossBackward0>)
tensor(2.8270, grad_fn=<NllLossBackward0>)
tensor(2.7184, grad_fn=<NllLossBackward0>)
tensor(2.6999, grad_fn=<NllLossBackward0>)
tensor(2.7440, grad_fn=<NllLossBackward0>)
tensor(2.7376, grad_fn=<NllLossBackward0>)
tensor(2.7733, grad_fn=<NllLossBackward0>)
tensor(2.8439, grad_fn=<NllLossBackward0>)
tensor(2.89

In [15]:
import torch.nn.functional as F

# Trying out the batch normalisation
epoch = 1

lookup_table = torch.randn((vocab_size, 2), requires_grad=True)
w1 = torch.randn((6, 100), requires_grad=True)
# b1 = torch.randn(100, requires_grad=True)
w2 = torch.randn((100, vocab_size), requires_grad=True)
b2 = torch.randn(vocab_size, requires_grad=True)

bngain = torch.ones((1, 100), requires_grad = False)
bnbias = torch.zeros((1, 100), requires_grad = False)
bngain_global = torch.ones((1, 100), requires_grad = False)
bnbias_global = torch.zeros((1, 100), requires_grad = False)

parameters = [lookup_table, w1, w2, b2, bngain, bnbias]

for p in parameters:
    p.requires_grad = True
    
for _ in range(epoch):
    embeddings = lookup_table[xs_enc]
    n1 = embeddings.view(-1, 6) @ w1 # + b1

    bn1 = bngain * ((n1 - torch.mean(n1, dim=0, keepdim=True)) / torch.std(n1, dim=0, keepdim=True)) + bnbias
    with torch.no_grad():
        bngain_global = 0.999 * bngain_global + 0.001 * bngain
        bnbias_global = 0.999 * bnbias_global + 0.001 * bnbias

    h1 = torch.tanh(bn1)
    logits = h1 @ w2 + b2
    loss = F.cross_entropy(logits, ys_enc)
    loss.backward()
    for p in parameters:
        p.data += -0.1 * p.grad
        p.grad = None
    print(loss)

tensor(18.2311, grad_fn=<NllLossBackward0>)


In [20]:
# Let's write everything properly
from collections import deque
import torch
import math
import torch.nn.functional as F


context = 3


def load_data(data_file):
    with open(data_file, 'r') as file:
        lines = file.readlines()
    return lines

def get_vocab(lines):
    vocab = set()
    
    for line in lines:
        if line.strip() != "":
            line = '<sos>' + line.strip() + '<eos>'
            for ch in line:
                vocab.add(ch)
    vocab = list(set(vocab))
    return vocab

def get_lookup_hashes(vocab):
    word_to_i = {word: i for i, word in enumerate(vocab)}
    i_to_word = {i: word for word, i in word_to_i.items()}
    return word_to_i, i_to_word

def prepare_dataset(lines):
    X = []
    Y = []

    X_enc = []
    Y_enc = []

    def get_queue():
        qu = deque(maxlen=context)
        for _ in range(context):
            qu.append('.')
        return qu
    
    for line in lines:
        qu = get_queue()
        if line.strip() != "":
            line = line.strip()
            qu.append(line[0])
            for ch in line[1:]:
                X.append(list(qu))
                Y.append(ch)
                qu.append(ch)

    for inp, out in zip(X, Y):
        temp = []
        for i in inp:
            temp.append(word_to_i[i])
        X_enc.append(temp)
        Y_enc.append(word_to_i[out])

    X = torch.tensor(X_enc)
    Y = torch.tensor(Y_enc)

    return X, Y

def get_data_splits(split1, split2, X, Y):
    X_train = X[:split1]
    X_validation = X[split1:split2]
    X_test = X[split2:]

    Y_train = Y[:split1]
    Y_validation = Y[split1:split2]
    Y_test = Y[split2:]
    
    return [(X_train, Y_train), (X_validation, Y_validation), (X_test, Y_test)]

vocab_size = len(vocab)
embedding_size = 2
hidden_layer_size = 100
learning_rate = 0.1

lookup_table = torch.randn((vocab_size, embedding_size), requires_grad=True)
w1 = torch.randn((context * embedding_size, hidden_layer_size), requires_grad=True)
# b1 = torch.randn(hidden_layer_size, requires_grad=True)
w2 = torch.randn((hidden_layer_size, vocab_size), requires_grad=True)
b2 = torch.randn(vocab_size, requires_grad=True)
bngain = torch.ones((1, 100), requires_grad = True)
bnbias = torch.zeros((1, 100), requires_grad = True)
bngain_global = torch.ones((1, 100), requires_grad = False)
bnbias_global = torch.zeros((1, 100), requires_grad = False)

parameters = [lookup_table, w1, w2, b2, bngain, bnbias]

def forward_pass(X, Y, parameters, learning_rate, epoch, batch_size, is_training=True):
    logits = None
    loss = None
    for _ in range(epoch):
        if not is_training:
            sampling = torch.randint(0, X.shape[0], (batch_size,))
            X_batch = X[sampling]
            Y_batch = Y[sampling]
        else:
            X_batch = X
            Y_batch = Y

        embeddings = lookup_table[X_batch]
        n1 = embeddings.view(-1, context * embedding_size) @ w1 #+ b1
        if is_training:
            bn1 = bngain * ((n1 - torch.mean(n1, dim=0, keepdim=True)) / torch.std(n1, dim=0, keepdim=True)) + bnbias
            with torch.no_grad():
                global bngain_global
                global bnbias_global
                bngain_global = 0.999 * bngain_global + 0.001 * bngain
                bnbias_global = 0.999 * bnbias_global + 0.001 * bnbias
        else:
            bn1 = bngain_global * ((n1 - torch.mean(n1, dim=0, keepdim=True)) / torch.std(n1, dim=0, keepdim=True)) + bnbias_global

        h1 = torch.tanh(bn1)
        logits = h1 @ w2 + b2

        if is_training:
            loss = F.cross_entropy(logits, Y_batch)
            loss.backward()
            for p in parameters:
                p.data += -learning_rate * p.grad
                p.grad = None
    
    
    return logits, loss

# k here is number of lines from inoput file
k = 10000
data_file = f"./data/mr_{k}.txt"

lines = load_data(data_file)
vocab = get_vocab(lines)
word_to_i, i_to_word = get_lookup_hashes(vocab)
X, Y = prepare_dataset(lines)

num_examples = X.shape[0]

[(X_train, Y_train), (X_validation, Y_validation), (X_test, Y_test)] = get_data_splits(math.ceil(0.8*num_examples), math.ceil(0.9*num_examples), X, Y)

In [23]:
# Let's train
_, loss = forward_pass(X_train, Y_train, parameters, learning_rate, epoch=1000, batch_size=1000, is_training=True)
print(loss)

tensor(3.1411, grad_fn=<NllLossBackward0>)


In [ ]:
# Let's train with reduced learning rate
_, loss = forward_pass(X_train, Y_train, parameters, learning_rate=0.01, epoch=100, batch_size=1000, is_training=True)
print(loss)